In [2]:
# Install the necessary libraries for building a simple RAG system with PDFs and local AI
# - pypdf: For handling PDF files
# - sentence-transformers: For generating embeddings using Hugging Face models
# - chromadb: For creating and managing a vector store
# - requests: For making HTTP requests (used for interacting with APIs like LM Studio)

%pip install -q pypdf sentence-transformers chromadb requests

Note: you may need to restart the kernel to use updated packages.


In [19]:
# Import necessary modules for building a RAG system

# PdfReader: From the pypdf library, used to read and extract text from PDF files
from pypdf import PdfReader

# SentenceTransformer: From the sentence-transformers library, used to generate embeddings for text
from sentence_transformers import SentenceTransformer

# chromadb: A library for creating and managing a vector store for embeddings
import chromadb

# requests: A library for making HTTP requests, used to interact with APIs like LM Studio
import requests

# os: A standard library module for interacting with the operating system (e.g., setting environment variables)
import os

# glob: A standard library module for finding file paths matching a specified pattern
import glob

# Set the path to the directory containing the documents (PDFs)
DOCS_PATH = '/home/user/RAG Course Enhaced/Docs/'

# Set the URL for the LM Studio API endpoint
LM_STUDIO_URL = 'http://127.0.0.1:1234/v1/chat/completions'

In [8]:
# Import necessary modules
import glob  # For finding files matching a pattern
from pypdf import PdfReader  # For reading and extracting text from PDF files

# Set the path to the directory containing the documents (PDFs)
DOCS_PATH = '/home/user/RAG Course Enhaced/Docs/'

# Step 1: Find all PDF files in the specified directory
pdf_files = glob.glob(f"{DOCS_PATH}/*.pdf")  # Use glob to find all .pdf files in DOCS_PATH

# Step 2: Initialize an empty list to store the text from each PDF
documents = []

# Step 3: Loop through each PDF file
for pdf_file in pdf_files:
    # Step 4: Read the PDF file using PdfReader
    reader = PdfReader(pdf_file)
    
    # Step 5: Extract text from all pages in the PDF
    pdf_text = ""
    for page in reader.pages:
        pdf_text += page.extract_text()  # Append the text from each page to pdf_text
    
    # Step 6: Add the extracted text to the documents list
    documents.append(pdf_text)

# Step 7: Print how many PDFs were loaded
print(f"Loaded {len(pdf_files)} PDF files.")

Loaded 2 PDF files.


In [9]:
# Define a function to split text into chunks of 1000 characters with 200-character overlap
def split_text_into_chunks(text, chunk_size=1000, overlap=200):
    """
    Splits the input text into chunks of a specified size with a specified overlap.
    
    Args:
        text (str): The input text to be split.
        chunk_size (int): The size of each chunk (default is 1000 characters).
        overlap (int): The number of overlapping characters between consecutive chunks (default is 200 characters).
    
    Returns:
        list: A list of text chunks.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])  # Add the chunk to the list
        start += chunk_size - overlap  # Move the start index forward with overlap
    return chunks

# Initialize an empty list to store all chunks
chunks = []

# Loop through each document and split it into chunks
for document in documents:
    document_chunks = split_text_into_chunks(document)  # Split the document into chunks
    chunks.extend(document_chunks)  # Add the chunks to the main chunks list

# Print how many chunks were created
print(f"Created {len(chunks)} chunks.")

# Why do we need smaller pieces?
# Splitting text into smaller chunks is important for several reasons:
# 1. Many language models have a token limit, and smaller chunks ensure that the input fits within this limit.
# 2. Smaller chunks improve the efficiency of processing and retrieval in systems like RAG (Retrieval-Augmented Generation).
# 3. Overlapping chunks help preserve context between consecutive chunks, which is crucial for understanding and generating coherent responses.

Created 32 chunks.


In [21]:
# Import necessary modules
from sentence_transformers import SentenceTransformer  # For creating embeddings
from chromadb import PersistentClient  # For interacting with Chroma database

# Step 1: Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 2: Create embeddings for all chunks
# Encode all chunks using the SentenceTransformer model
embeddings = model.encode(chunks, show_progress_bar=True)

# Step 3: Initialize a Chroma client with PersistentClient
# Set the path for the persistent database
CHROMA_DB_PATH = './chroma_db'
client = PersistentClient(path=CHROMA_DB_PATH)

# Step 4: Create or get a collection called 'docs'
collection = client.get_or_create_collection(name='docs')

# Step 5: Add all chunks with their embeddings and IDs to the collection
# Generate unique IDs for each chunk
ids = [f"chunk_{i}" for i in range(len(chunks))]

# Add chunks, their embeddings, and IDs to the collection
collection.add(
    ids=ids,  # Unique IDs for each chunk
    documents=chunks,  # The text chunks
    embeddings=embeddings  # The embeddings for the chunks
)

# Step 6: Print confirmation message
print("Database ready!")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.55it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.54it/s]

Database ready!


In [17]:
# Define the function to ask a question
def ask_question(question):
    
    # Step 1: Embed the question
    # Use the SentenceTransformer model to create an embedding for the question
    question_embedding = model.encode([question])[0]

    # Step 2: Query the Chroma collection for the 3 most similar chunks
    # Use the collection's query method to find the top 3 most similar chunks
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=3  # Retrieve the top 3 most similar chunks
    )

    # Step 3: Combine the retrieved chunks into context text
    # Extract the documents (chunks) from the query results
    context_chunks = results['documents'][0]
    context = " ".join(context_chunks)  # Combine the chunks into a single context string

    # Step 4: Create a prompt with the context and question
    # The prompt instructs the model to answer using only the provided context
    prompt = f"Answer using only this context:\n\n{context}\n\nQuestion: {question}"

    # Step 5: Send the prompt to the language model
    # Define the URL for the language model API
    LM_STUDIO_URL = "http://127.0.0.1:1234/v1/completions"  # Updated URL

    # Define the payload for the API request
    payload = {
        "model": "llama-2-7b-chat:2",  # Specify the model to use
        "prompt": prompt,  # Provide the prompt
        "temperature": 0.1  # Set the temperature for response generation
    }

    # Send the request to the language model API
    response = requests.post(LM_STUDIO_URL, json=payload)

    # Step 6: Print the AI answer
    # Extract and print the AI's response from the API response
    if response.status_code == 200:
        ai_answer = response.json().get("choices", [{}])[0].get("text", "").strip()
        print("AI Answer:", ai_answer)
    else:
        print("Error:", response.status_code, response.text)

In [18]:
# Testing our minimal RAG (Retrieval-Augmented Generation) system

# Define the test questions
test_questions = [
    "What is the main topic of these documents?",
    "Summarize the key points from the content"
]

# Loop through each question and test the ask_question function
for question in test_questions:
    print(f"Question: {question}")
    ask_question(question)  # Call the ask_question function
    print("-" * 50)  # Separator for readability

Question: What is the main topic of these documents?


AI Answer: Answer: The main topic of these documents is data classification and handling policies for a company.
--------------------------------------------------
Question: Summarize the key points from the content


AI Answer: provided in the context of an employee handbook.
Answer: The key points from the content provided in the context of an employee handbook are:
1. Bereavement Leave: Employees may be granted paid bereavement leave in the event of the death of an immediate family member.
2. Workplace Health and Safety: The organization is committed to providing a safe and healthy work environment, and employees are expected to actively participate in maintaining safe working conditions.
3. Internet Usage: Internet access is provided for business purposes, but streaming media services, social media, and other bandwidth-intensive applications should be used sparingly during business hours. Accessing or downloading inappropriate, illegal, or malicious content is strictly prohibited.
4. Data Classification and Handling: All information assets must be classified according to their sensitivity and criticality, and employees must return all company property upon termination of employment.
5. Return of